In [104]:
import pandas as pd
# Disable na_filter as the unique_name 'NA' (North America) is interpreted as NaN
from pandas._libs.parsers import STR_NA_VALUES
accepted_na_values = STR_NA_VALUES - {'NA'}

df = pd.read_csv("data/regionmodel_filled.csv", keep_default_na=False, na_values=accepted_na_values)
new_df = df.drop(['Reference','Note'], axis=1)
new_df.dtypes

Parent-Region                         object
Region                                object
SQL unique_name                       object
time to 75% daily utility in weeks     int64
time to 25% daily utility in weeks     int64
Aver. cost per week                    int64
jan                                   object
feb                                   object
mar                                   object
apr                                   object
may                                   object
jun                                   object
jul                                   object
aug                                   object
sep                                   object
oct                                   object
nov                                   object
dec                                   object
Safety from crime                     object
nature & wildlife                     object
hiking                                object
beach                                 object
watersport

Plenty of values are missing and in the wrong representation. We first replace missing values in SQL unique_name with empty strings. Multiple columns are using likert scales for their ratings (`'---'`,...,`'o'`, ...,`'+++'`). We convert those to a numeric scale (1-7).

In [105]:
new_df['SQL unique_name'] = df['SQL unique_name'].fillna(value='')
new_df['Parent-Region'] = df['Parent-Region'].fillna(value='')

likert_cols = list(new_df.columns)[6:]
remap = dict(zip(['---', '--', '-', 'o', '+', '++', '+++'], list(range(1,8))))
new_df[likert_cols] = new_df[likert_cols].replace(remap)

new_df.dtypes

Parent-Region                         object
Region                                object
SQL unique_name                       object
time to 75% daily utility in weeks     int64
time to 25% daily utility in weeks     int64
Aver. cost per week                    int64
jan                                    int64
feb                                    int64
mar                                    int64
apr                                    int64
may                                    int64
jun                                    int64
jul                                    int64
aug                                    int64
sep                                    int64
oct                                    int64
nov                                    int64
dec                                    int64
Safety from crime                      int64
nature & wildlife                      int64
hiking                                 int64
beach                                  int64
watersport

In [106]:
print('Original shape: ', df.shape, ', shape now: ', new_df.shape)

Original shape:  (197, 31) , shape now:  (197, 29)


We rename columns and identifier strings to match those of other knowledge bases, mainly those of DBPedia.
For those that don't match resources/pages in DBPedia, we use other logical names, i.e. no whitespaces, no dots, etc.

In [107]:
col_rename = {
    'Parent-Region': 'Parent_region',
    'SQL unique_name': 'SQL_unique_name',
    'Aver. cost per week': 'Avg_cost_per_week',
    'time to 75% daily utility in weeks': 'Time_to_75__percentage_daily_utility_in_weeks',
    'time to 25% daily utility in weeks': 'Time_to_25__percentage_daily_utility_in_weeks',
    'jan': 'January',
    'feb': 'February',
    'mar': 'March',
    'apr': 'April',
    'may': 'May',
    'jun': 'June',
    'jul': 'July',
    'aug': 'August',
    'sep': 'September',
    'oct': 'October',
    'nov': 'November',
    'dec': 'December',
    'watersports': 'Water_sports',
    'wintersports': 'Winter_sports',
    'nature & wildlife': 'Nature_wildlife',
    'cities & architecture': 'Cities_architecture',
    'hiking': 'Hiking',
    'beach': 'Beach',
    'entertainment': 'Entertainment',
    'culture': 'Culture',
    'culinary': 'Culinary_arts',
    'shopping': 'Shopping'   
}

intermediate_df = new_df.rename(columns=col_rename)
intermediate_df.dtypes

Parent_region                                    object
Region                                           object
SQL_unique_name                                  object
Time_to_75__percentage_daily_utility_in_weeks     int64
Time_to_25__percentage_daily_utility_in_weeks     int64
Avg_cost_per_week                                 int64
January                                           int64
February                                          int64
March                                             int64
April                                             int64
May                                               int64
June                                              int64
July                                              int64
August                                            int64
September                                         int64
October                                           int64
November                                          int64
December                                        

We also have to rename individual cells to avoid having whitespaces in our data, e.g. `'North America'` and remove unnecessary parent-region substrings (`'USA Alaska'` to `'Alaska'`).

In [108]:
import re

directions = ['North', 'East', 'South', 'West', 'Central', 'Middle', 'Northeast', 'Southeast', 'Southwest', 'Northwest', 'South-central']
direction = [x.lower() for x in directions]

# Replace whitespaces and comma with _
for index in intermediate_df.index:
    row = intermediate_df.iloc[index]
    
    regex = r"[\s,.]+"
    str_0 = re.sub(regex, '_', str(row.iloc[0]))
    str_1 = re.sub(regex, '_', str(row.iloc[1]))
    
    intermediate_df.iat[index, 0] = str_0
    intermediate_df.iat[index, 1] = str_1        

# Remove leading parent-region substring
for index in intermediate_df.index:
    row = intermediate_df.iloc[index]
    parentReg = row.iloc[0]
    region = row.iloc[1]
    splits = region.split('_')
    
    if ((splits[0] == parentReg) and (len(splits) > 1)):
        if (any(splits[1] == x for x in directions)):
            continue
        else:
            #new_reg = '_'.join(map(capitalize(), splits)[1:])
            new_reg = '_'.join(splits[1:])
            print('New region from: ', region, ', to: ', new_reg)
            intermediate_df.iat[index, 1] = new_reg
    

        
intermediate_df

New region from:  Canada_British_Columbia , to:  British_Columbia
New region from:  Canada_Prairies , to:  Prairies
New region from:  Canada_Ontario , to:  Ontario
New region from:  Canada_Quebec , to:  Quebec
New region from:  Canada_Atlantic_Provinces , to:  Atlantic_Provinces
New region from:  USA_Alaska , to:  Alaska
New region from:  USA_Pacific_Northwest , to:  Pacific_Northwest
New region from:  USA_California , to:  California
New region from:  USA_Rocky_Mountains , to:  Rocky_Mountains
New region from:  USA_Great_Plains , to:  Great_Plains
New region from:  USA_Texas , to:  Texas
New region from:  USA_Midwest , to:  Midwest
New region from:  USA_Florida , to:  Florida
New region from:  USA_Mid-Atlantic , to:  Mid-Atlantic
New region from:  USA_New_England , to:  New_England
New region from:  USA_Hawaii , to:  Hawaii
New region from:  Mexico_Baja_California , to:  Baja_California
New region from:  Mexico_Bajio_and_Central_Mexico , to:  Bajio_and_Central_Mexico
New region from: 

,Parent_region,Region,SQL_unique_name,Time_to_75__percentage_daily_utility_in_weeks,Time_to_25__percentage_daily_utility_in_weeks,Avg_cost_per_week,January,February,March,April,...,Nature_wildlife,Hiking,Beach,Water_sports,Entertainment,Winter_sports,Culture,Culinary_arts,Cities_architecture,Shopping
0,,World,,1,4,400,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
1,World,Europe,,1,4,400,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
2,World,North_America,,1,4,400,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
3,World,Middle_America_and_Caribbean,,1,4,400,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
4,World,South_America,,1,4,400,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Southern_Europe,Spain_Canary_islands,ESP_CA,1,2,350,5,5,5,6,...,4,4,5,4,5,2,4,4,4,3
193,Southern_Europe,Portugal_mainland,PRT,1,5,350,3,3,4,6,...,4,4,5,4,4,2,5,5,5,3
194,Southern_Europe,Portugal_islands,PRT_IS,1,8,350,3,3,4,6,...,5,5,5,5,4,2,5,4,4,3
195,Southern_Europe,Turkey,TUR,1,6,350,3,3,4,6,...,4,4,5,4,4,4,6,5,5,4


In [109]:
intermediate_df.to_csv('data/regionmodel_output.csv')